In [ ]:
"""
@Authors:                   @Email:
* Shaiful                     shaiful019@gmail.com
* Rezwanul                    rezwanul.cse@gmail.com
* Oli
* Habibul
* Tahmid
"""

# Problem: Return to Origin (RTO) Customers Prediction
 Problem class: Supervised Classification
 
 Problem dataset link: Real dataset so no online link.
 
 Problem description: 
 
     RTO (Return to Origin) is a Classification Problem in eCommerce platforms where Many customers cancel their order when the   product is already on shipping. Then customer don't response and the product is return back to office.
 
 Problem Task:
     Have to predict who will Cancel the product order after product is already on shipping.

# Importing libraries

In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

# Feature or column description

COLUMN DESCRIPTIONS:
* Order Id
* User Id
* Order Value (In BDT, amount of the order)
* Payment Type (How, customers paid for it. MPD/MPC = Manual, means Cash on delivery. MPS = Bkash, Rocket type mobile payments. * OPS = online credit card, debit card. ADC = AD Cash (can be earned, can be recharged), EMI = Monthly EMI Purchase )
* District (Where customers are getting delivered)
* OrderSource (Android, Mobile Site or Desktop Site)
* Order Type for Delivery ( Express = Quick delivery and paid more for it, Normal = average delivery)
* Delivery Charge ( for the delivery)
* Order Placing Date (When customer placed the order)
* Order Confirmation date (Office make calls to confirm the order. It call is made late, they might not want the product later)
* IsCartOrder (is it a cart order or single product order)
* OrderFromPromotionalEvent (Is product purchased in discount from a promotional event or not)
* courierService (Courier service used by the customer)
* isRTO ( Order canceled after product is already on shipping )